In [2]:
import tensorflow as tf
import datetime
%load_ext tensorboard

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
    return tf.keras.models.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28,28)),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10, activation='softmax')
        ]
    )

In [4]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "log/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(
    x=x_train,
    y=y_train,
    epochs=5,
    validation_data=(x_test, y_test),
    callbacks=[tensorboard_callback]
)

2022-04-05 10:39:56.638927: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-05 10:39:56.639104: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro
Epoch 1/5


2022-04-05 10:39:56.855141: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-05 10:39:57.014308: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1865/1875 [============================>.] - ETA: 0s - loss: 0.2201 - accuracy: 0.9345

2022-04-05 10:40:06.645589: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 11s 5ms/step - loss: 0.2197 - accuracy: 0.9346 - val_loss: 0.1227 - val_accuracy: 0.9620
Epoch 2/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0951 - accuracy: 0.9714 - val_loss: 0.0858 - val_accuracy: 0.9734
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0673 - accuracy: 0.9791 - val_loss: 0.0710 - val_accuracy: 0.9787
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0520 - accuracy: 0.9839 - val_loss: 0.0694 - val_accuracy: 0.9784
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0424 - accuracy: 0.9860 - val_loss: 0.0731 - val_accuracy: 0.9784


In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [8]:
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [9]:
def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)

In [10]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'log/gradient_tape/' + current_time + '/train'
test_log_dir = 'log/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [12]:
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.2438182234764099, Accuracy: 92.84333801269531, Test Loss: 0.12197967618703842, Test Accuracy: 95.9800033569336
Epoch 2, Loss: 0.10297495871782303, Accuracy: 96.92333221435547, Test Loss: 0.08293206989765167, Test Accuracy: 97.3800048828125
Epoch 3, Loss: 0.0721706748008728, Accuracy: 97.8133316040039, Test Loss: 0.07451942563056946, Test Accuracy: 97.74000549316406
Epoch 4, Loss: 0.05512937530875206, Accuracy: 98.25666809082031, Test Loss: 0.06872274726629257, Test Accuracy: 97.81000518798828
Epoch 5, Loss: 0.04280401021242142, Accuracy: 98.58333587646484, Test Loss: 0.06015503779053688, Test Accuracy: 98.02000427246094
Epoch 1, Loss: 0.18748517334461212, Accuracy: 94.31000518798828, Test Loss: 0.09626999497413635, Test Accuracy: 97.04000091552734
Epoch 2, Loss: 0.08253485709428787, Accuracy: 97.43500518798828, Test Loss: 0.06996666640043259, Test Accuracy: 97.7800064086914
Epoch 3, Loss: 0.0583597868680954, Accuracy: 98.21666717529297, Test Loss: 0.06354180723428726, 